In [1]:
# -*- coding: utf-8 -*-
# Developed: Aug 2023
# Last update: Dec 2023
'''
Procedure for K0 search.
'''

import os
import shutil
import datetime as dt
import time
import glob
import numpy as np
import pickle
from JobCreation import Load_HeliosphericParameters, griglia_valori_k0, SubmitSims
from EvaluateSimAndFlux import GetRawMatrix, EvaluateFlux

DEBUG = True
NK0 = 400
SLEEP_TIME = 60
APP = "_FD006"
OUTPUT_DICT = {}

# TODO: go to the file JobCreation.py and change the value of SIMROOTDIR to the correct path

# Function Definitions
def load_simulation_list(app_name, debug=False):
    """
        Load the list of simulations.

        Parameters:
        app_name (str): the application name.
        debug (bool): whether to print debug information.

        Returns:
        sim_list (list): the list of simulations.
    """

    # Counters for excluded simulations (Added by me)
    excluded_sims_filter = 0
    excluded_sims_type = 0

    sim_list = []
    if debug:
        print(f"Loading simulations for application: {app_name}")
        
    for line in open(f"Simulations{app_name}.list").readlines():
        if line.startswith("#"):
            continue
        
        single_sim = [x.strip() for x in line.replace("\t", "").split("|")[:8]]
        if debug:
            print(f"Parsed simulation: {single_sim}")

        # if debug is true, it excludes all files obtaining a final empty list, so I inibited it (MG)
        # if debug:
        if False:
            if (float(single_sim[3]) < 20180101 or float(single_sim[3]) > 20180102):
                excluded_sims_filter += 1
            continue

        if "Electron" in single_sim[1] or "Positron" in single_sim[1]:
            if debug:
                excluded_sims_type += 1
            continue
        
        print(f"Adding simulation: {single_sim}")
        sim_list.append(single_sim)

        ions = single_sim[1].strip()
        if ions not in OUTPUT_DICT:
            OUTPUT_DICT[ions] = {}
            if debug:
                print(f"Added new ion type to OUTPUT_DICT: {ions}")

    if debug:
        print(f"Loaded simulation list:")
        for sim in sim_list:
            print(sim)
        print(f"Ecluded simulations due to filter: {excluded_sims_filter}")
        print(f"Ecluded simulations due to particle type: {excluded_sims_type}")

    return sim_list

def initialize_output_dict(sim_list, x_exp, f_exp, er_exp_inf, er_exp_sup, k0_ref, k0_ref_err, debug=False):
    """
        Initialize the output dictionary with experimental data.

        Parameters:
        sim_list (list): the list of simulations.
        x_exp (np.array): the experimental rigidity values.
        f_exp (np.array): the experimental flux values.
        er_exp_inf (np.array): the experimental flux lower error values.
        er_exp_sup (np.array): the experimental flux upper error values.
        k0_ref (float): the reference K0 value.
        k0_ref_err (float): the reference K0 error value.
        debug (bool): whether to print debug information.

        Returns:
        None
    """

    # debug = True

    for el in sim_list:
        ions = el[1].strip()
        time = dt.datetime.strptime(el[3], '%Y%m%d').date()
        if debug:
            print(f"Processing simulation for ion: {ions}, date: {time}")

        # For each rigidity value in the experimental data (in the range [3, 11])
        for i, T in enumerate(x_exp):
            if not (3 <= T <= 11):
                continue

            if debug:
                print(f"Processing rigidity value: {T}")

            # Initialize the output dictionary
            if T not in OUTPUT_DICT[ions]:
                OUTPUT_DICT[ions][T] = {}
                if debug:
                    print(f"Initialized OUTPUT_DICT for ion: {ions}, rigidity: {T}")

            # Initialize the output dictionary for the current ion and rigidity value
            OUTPUT_DICT[ions][T][time] = {
                "diffBest": 9e99,
                "K0best": 0,
                "K0Min": 9e99,
                "K0Max": 0,
                "Fluxbest": 0,
                "FluxMin": 9e99,
                "FluxMax": 0,
                "fExp": f_exp[i],
                "ErExp_inf": er_exp_inf[i],
                "ErExp_sup": er_exp_sup[i],
                "K0ref": k0_ref,
                "K0Err_ref": k0_ref_err * k0_ref,
            }
            if debug:
                print(f"Initialized data for ion: {ions}, rigidity: {T}, date: {time}")

    if debug:
        print("Final OUTPUT_DICT:")
        for ion, data in OUTPUT_DICT.items():
            print(f"Ion: {ion}, Data: {data}")


def wait_for_files(output_dir_path_name, n_files_expected, sleep_time, debug=False):
    """
        Wait until the expected number of output files is present.

        Parameters:
        output_dir_path_name (str): the path to the output directory.
        n_files_expected (int): the number of files expected.
        sleep_time (int): the time to sleep between checks.
        debug (bool): whether to print debug information.

        Returns:
        None
    """
    print(f"Waiting for {n_files_expected} files in {output_dir_path_name}...")
    while len(glob.glob(output_dir_path_name)) < n_files_expected:
        time.sleep(sleep_time)
        if debug:
            print(f"Found {len(glob.glob(output_dir_path_name))} files so far.")

def process_simulations(el_sim_list, raw_matrix_sims, x_exp, debug=False):
    """
        Process simulation results to find the best K0 values.

        Parameters:
        el_sim_list (list): the list of simulations.
        raw_matrix_sims (dict): the raw matrix of simulation results.
        x_exp (np.array): the experimental rigidity values.
        debug (bool): whether to print debug information.

        Returns:
        None
    """
    ions = el_sim_list[1].strip()
    time = dt.datetime.strptime(el_sim_list[3], '%Y%m%d').date()
    if debug:
        print(f"Processing simulations for ion: {ions}, date: {time}")

    fluxes = EvaluateFlux(el_sim_list, raw_matrix_sims, x_exp, debug)

    # For each K0 value and simulated flux value
    for k0, (sim_en_rig, sim_flux) in fluxes.items():
        if debug:
            print(f"Evaluating K0: {k0}")

        # For each rigidity value in the experimental data (in the range [3, 11])
        for i, F in enumerate(sim_flux):
            T = x_exp[i]
            if not (3 <= T <= 11):
                continue

            ref_entry = OUTPUT_DICT[ions][T][time]
            diff_val = F - ref_entry['fExp']
            if debug:
                print(f"Rigidity: {T}, Simulated Flux: {F}, Experimental Flux: {ref_entry['fExp']}, Difference: {diff_val}")

            if -ref_entry['ErExp_inf'] <= diff_val <= ref_entry['ErExp_sup']:
                if debug:
                    print(f"Difference {diff_val} is within error bounds [{-ref_entry['ErExp_inf']}, {ref_entry['ErExp_sup']}]")

                # If the difference is better than the current best difference, update the reference entry
                if abs(diff_val) < abs(ref_entry['diffBest']):
                    ref_entry['diffBest'] = diff_val
                    ref_entry['K0best'] = k0
                    ref_entry['Fluxbest'] = F
                    if debug:
                        print(f"Updated best values for ion: {ions}, rigidity: {T}, date: {time}")

                # If the K0 value is better than the current best K0 value, update the reference entry
                if ref_entry['K0Min'] > k0:
                    ref_entry['FluxMin'] = F
                    ref_entry['K0Min'] = k0
                    if debug:
                        print(f"Updated minimum K0 for ion: {ions}, rigidity: {T}, date: {time}")

                # If the K0 value is better than the current worst K0 value, update the reference entry
                if ref_entry['K0Max'] < k0:
                    ref_entry['FluxMax'] = F
                    ref_entry['K0Max'] = k0
                    if debug:
                        print(f"Updated maximum K0 for ion: {ions}, rigidity: {T}, date: {time}")

# EXECUTION FLOW
# Load simulation list
sim_list = load_simulation_list(APP, DEBUG)
# Load heliospheric parameters
hpar = Load_HeliosphericParameters(DEBUG)

for el_sim_list in sim_list:
    print(f"Processing {el_sim_list[0]} {el_sim_list[1]}...")

    # Create grid of K0 values to simulate
    k0_arr, k0_ref, k0_ref_err = griglia_valori_k0(el_sim_list[3], NK0, hpar, DEBUG)
    if DEBUG:
        print(f"Generated K0 grid: {k0_arr}, reference K0: {k0_ref}, reference K0 error: {k0_ref_err}")

    # Load experimental data and initialize output dictionary
    file_name = f"DataTXT/{el_sim_list[2]}"
    x_exp, f_exp, er_exp_inf, er_exp_sup = np.loadtxt(file_name, unpack=True, usecols=(0, 1, 2, 3))
    valid_indices = (x_exp >= 3) & (x_exp <= 11)
    x_exp, f_exp = x_exp[valid_indices], f_exp[valid_indices]
    er_exp_inf, er_exp_sup = er_exp_inf[valid_indices], er_exp_sup[valid_indices]
    if DEBUG:
        print(f"Loaded experimental data from {file_name}")

    initialize_output_dict([el_sim_list], x_exp, f_exp, er_exp_inf, er_exp_sup, k0_ref, k0_ref_err, debug=DEBUG)

    # Submit simulations and wait for results
    output_dir_path, output_file_list = SubmitSims(el_sim_list, k0_arr, R_range=[3, 11])
    
    if DEBUG:
        print(f"Submitted simulations, output directory: {output_dir_path}, output files: {output_file_list}")

    wait_for_files(f"{output_dir_path}/outfile/*.dat", len(output_file_list), SLEEP_TIME, DEBUG)
    if DEBUG:
        print(f"All simulation files are ready in {output_dir_path}/outfile/")

    # Process results of simulations
    raw_matrix_sims = GetRawMatrix(output_dir_path, output_file_list, DEBUG)
    process_simulations(el_sim_list, raw_matrix_sims, x_exp, debug=DEBUG)

    # Cleanup output directory
    if output_dir_path and 'Simulations_K0Search' in output_dir_path:
        print(f"Removing directory {output_dir_path}.")
        shutil.rmtree(output_dir_path)

# Save results to file Result{APP}.pkl using pickle
with open(f"Result{APP}.pkl", "wb") as f:
    pickle.dump(OUTPUT_DICT, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Results saved to Result{APP}.pkl")


Loading simulations for application: _FD006
Parsed simulation: ['AMS-02Daily_20110801', 'Helium', 'Rigidity_AMS-02Daily_20110801_Helium.dat', '20110801', '20110801', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110801', 'Helium', 'Rigidity_AMS-02Daily_20110801_Helium.dat', '20110801', '20110801', '1', '0', '0']
Added new ion type to OUTPUT_DICT: Helium
Parsed simulation: ['AMS-02Daily_20110802', 'Helium', 'Rigidity_AMS-02Daily_20110802_Helium.dat', '20110802', '20110802', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110802', 'Helium', 'Rigidity_AMS-02Daily_20110802_Helium.dat', '20110802', '20110802', '1', '0', '0']
Parsed simulation: ['AMS-02Daily_20110803', 'Helium', 'Rigidity_AMS-02Daily_20110803_Helium.dat', '20110803', '20110803', '1', '0', '0']
Adding simulation: ['AMS-02Daily_20110803', 'Helium', 'Rigidity_AMS-02Daily_20110803_Helium.dat', '20110803', '20110803', '1', '0', '0']
Parsed simulation: ['AMS-02Daily_20110804', 'Helium', 'Rigidity_AMS-02Daily_20110804_Helium.

KeyboardInterrupt: 